In [1]:
import torch
import numpy as np
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
#import the data
from htru1 import HTRU1

#Transforms the data. This seems like a required step as I kepts getting errors if I did not include it
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

#Specifically import the training data and test data
train_data = HTRU1('data', train=True, download=True, transform=transform)
test_data = HTRU1('data', train=False, download=True, transform=transform)

#Variable setup to extract the data
trainloader = torch.utils.data.DataLoader(train_data, batch_size=50000, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=True, num_workers=2)

#Extracts the data into the features and the labels
dataiter = iter(trainloader)
train_tensor_images, train_tensor_labels = dataiter.next()
dataiter = iter(testloader)
test_tensor_images, test_tensor_labels = dataiter.next()

#Converts the data to np arrays (found them easier to work with)
train_features = np.array(train_tensor_images)
train_label = np.array(train_tensor_labels)
test_features = np.array(test_tensor_images)
test_label = np.array(test_tensor_labels)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#transformed the feates from (5000, 3, 32, 32) into (3, 5000, 32, 32)
train_features_t = np.transpose(train_features, (1,0,2,3))
test_features_t = np.transpose(test_features, (1,0,2,3))

In [4]:
#Assigning each layer into their own variable
train_layer1 = train_features_t[0]
train_layer2 = train_features_t[1]
train_layer3 = train_features_t[2]
test_layer1 = test_features_t[0]
test_layer2 = test_features_t[1]
test_layer3 = test_features_t[2]

In [5]:
#Flattening the data
ndims = train_layer1.shape[1] * train_layer1.shape[2]
X_train = train_layer1.reshape(train_layer1.shape[0],ndims)
X_test = test_layer1.reshape(test_layer1.shape[0],ndims)

In [6]:
X_train_2 = train_layer1.reshape(train_layer2.shape[0],ndims)
X_test_2 = test_layer1.reshape(test_layer2.shape[0],ndims)

X_train_3 = train_layer1.reshape(train_layer3.shape[0],ndims)
X_test_3 = test_layer1.reshape(test_layer3.shape[0],ndims)

In [7]:
#renaming the y data
y_train = train_label
y_test  = test_label

In [8]:
#Importing tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
def build_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [13]:
def compile_model(model,metric):
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[metric])
    return model

In [18]:
def fit_model(model,epochs):
    model.fit(
    X_train_3,
    y_train,
    epochs=epochs,
    shuffle=True,
    verbose=1
    )
    return model

In [25]:
def run_model(metric,epoch):
    built_model = build_model()
    compiled_model = compile_model(built_model,metric)
    model = fit_model(compiled_model,epoch)
    y_pred = model.predict(X_test).round()
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    TPR = cm[0][0]/(cm[0][0]+cm[1][0])
    print((TPR.round(2))*100)
    accuracy_score(y_test, y_pred)
    print(accuracy_score)
    #return model

In [32]:
#y_pred = run_model('accuracy',10)
run_model('accuracy',10)

Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0662 - accuracy: 0.9829
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0443 - accuracy: 0.9882
Epoch 3/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0370 - accuracy: 0.9898
Epoch 4/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0331 - accuracy: 0.9907
Epoch 5/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0318 - accuracy: 0.9910
Epoch 6/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0302 - accuracy: 0.9911
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0297 - accuracy: 0.9910
Epoch 8/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0281 - accuracy: 0.9918
Epoch 9/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0285 - accuracy: 0.9917
Epoch 10/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.027

In [27]:
run_model('accuracy',25)

Epoch 1/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0639 - accuracy: 0.9833
Epoch 2/25
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0443 - accuracy: 0.9878
Epoch 3/25
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0387 - accuracy: 0.9892
Epoch 4/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0350 - accuracy: 0.9898
Epoch 5/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0323 - accuracy: 0.9903
Epoch 6/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0318 - accuracy: 0.9911
Epoch 7/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0300 - accuracy: 0.9913
Epoch 8/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0286 - accuracy: 0.9910
Epoch 9/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0268 - accuracy: 0.9918
Epoch 10/25
1563/1563 [==============================] - 4s 2ms/step - loss: 0.027

In [28]:
run_model('Recall',15)

Epoch 1/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0642 - accuracy: 0.9840
Epoch 2/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0443 - accuracy: 0.9881
Epoch 3/15
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0374 - accuracy: 0.9888
Epoch 4/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0356 - accuracy: 0.9899
Epoch 5/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0336 - accuracy: 0.9903
Epoch 6/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0309 - accuracy: 0.9911
Epoch 7/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0298 - accuracy: 0.9911
Epoch 8/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0292 - accuracy: 0.9915
Epoch 9/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0278 - accuracy: 0.9915
Epoch 10/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.026

In [29]:
run_model('TruePositives',15)

Epoch 1/15
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0672 - accuracy: 0.9829
Epoch 2/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0440 - accuracy: 0.9879
Epoch 3/15
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0383 - accuracy: 0.9894
Epoch 4/15
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0333 - accuracy: 0.9905
Epoch 5/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0330 - accuracy: 0.9907
Epoch 6/15
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0306 - accuracy: 0.9911
Epoch 7/15
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0284 - accuracy: 0.9916
Epoch 8/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0286 - accuracy: 0.9911
Epoch 9/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0273 - accuracy: 0.9918
Epoch 10/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.028

In [30]:
run_model('Precision',15)

Epoch 1/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0673 - accuracy: 0.9797
Epoch 2/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0497 - accuracy: 0.9801
Epoch 3/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0432 - accuracy: 0.9839
Epoch 4/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0385 - accuracy: 0.9893
Epoch 5/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0330 - accuracy: 0.9900
Epoch 6/15
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0314 - accuracy: 0.9908
Epoch 7/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0309 - accuracy: 0.9908
Epoch 8/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0294 - accuracy: 0.9911
Epoch 9/15
1563/1563 [==============================] - 4s 2ms/step - loss: 0.0270 - accuracy: 0.9912
Epoch 10/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.027